In [1]:
## load data

In [2]:
!pip install datasets
!pip install transformers
!pip install accelerate -U
!pip install evaluate
!pip install --upgrade -q wandb

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00


In [3]:
# dataset_path = "/MyDrive/Colab_Notebooks/CSCI-567_project/data/Topic_data/topic_data.csv"
dataset_path = "/kaggle/input/ad-data/ad_data.csv"

In [4]:
import csv
from datasets import Dataset, Image
# from charset_normalizer import md__mypyc

def load_data(data_path):
    csv_data_list = []
    with open(data_path, encoding="UTF-8") as f:
        reader = csv.reader(f)
        header_row = next(reader)
        # print(header_row)

        for row in reader:
            csv_data_list.append(row)

    # imgs = []
    # label_ids = []
    # labels = []
    path_label_dict = {}
    dataset_dict = {'img': [], 'label_id': []}  # "image": path,
    id2label_dict = {}
    label2id_dict = {}


    for row in csv_data_list:
#         img_path = "/kaggle/input/ad-data/" + row[0]
        label_id = int(row[1])
        if label_id == 0:
            img_path = "/kaggle/input/ad-data/resnet_negative/" + row[0]
        else:
            img_path = "/kaggle/input/csci-567/" + row[0]
        # label_ids.append(label_id)
        dataset_dict['label_id'].append(int(label_id))

        label_name = row[2]
        if label_id not in id2label_dict.keys():
            id2label_dict[label_id] = label_name
        
        if label_name not in label2id_dict.keys():
            label2id_dict[label_name] = label_id;
        # labels.append(label_name)

        # img_file = Image.open(img_path)
        # img_array = np.array(img_file)
        dataset_dict['img'].append(img_path)



        path_label_dict[img_path] = label_id
        # print(img_array.shape)

    return_dataset = Dataset.from_dict(dataset_dict).cast_column("img", Image())

    return return_dataset, id2label_dict, label2id_dict

whole_set, id2label, label2id = load_data(dataset_path)
splits = whole_set.train_test_split(test_size=0.2)
train_val_set = splits['train']
test_set = splits['test']
splits = train_val_set.train_test_split(test_size=0.25)
train_set = splits['train']
val_set = splits['test']
num_labels = len(id2label)

# print(len(whole_set))
# print(len(train_set))
# print(len(val_set))
# print(len(test_set))
print(type(train_set))

<class 'datasets.arrow_dataset.Dataset'>


In [5]:
# train_set = train_set.train_test_split(test_size=0.01)['test']


# print(type(train_set['label_id']))
# temp_list = list(id2label.keys())
# temp_list.sort()
# print(temp_list)

In [6]:
print(type(train_set))
print(len(train_set))
print(num_labels)

<class 'datasets.arrow_dataset.Dataset'>
13167
2


In [7]:
print(id2label)
print(label2id)

{1: 'ad', 0: 'not_ad'}
{'ad': 1, 'not_ad': 0}


## Preprocessing

In [8]:
from transformers import ViTImageProcessor, BeitImageProcessor

processor = BeitImageProcessor.from_pretrained('microsoft/beit-base-patch16-224', num_labels = num_labels)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor,
                                    RandomRotation)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size
# size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)
transform_train = Compose(
        [
            Resize([size["height"], size["width"]]),
#             RandomResizedCrop(size),
            RandomHorizontalFlip(),
            RandomRotation(30),
            ToTensor(),
            normalize,
        ]
    )

transform_val = Compose(
        [
            Resize([size["height"], size["width"]]),
            # CenterCrop(size),
            ToTensor(),
            normalize,
        ]
    )

def train_transforms(examples):
    examples['pixel_values'] = [transform_train(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [transform_val(image.convert("RGB")) for image in examples['img']]
    return examples

In [10]:
print(size)

{'height': 224, 'width': 224}


In [11]:
train_set.set_transform(train_transforms)
val_set.set_transform(val_transforms)
test_set.set_transform(val_transforms)

## Define the model

In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(device)

cuda


In [13]:
from transformers import BeitForImageClassification

model = BeitForImageClassification.from_pretrained\
        ('microsoft/beit-base-patch16-224',
         id2label=id2label,
         label2id=label2id,
         ignore_mismatched_sizes=True,
         num_labels = num_labels)

model = model.to(device)

Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import torch
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    # labels = torch.tensor([example["label"] for example in examples])
    labels = torch.tensor([example["label_id"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [15]:
from transformers import TrainingArguments, Trainer

metric_name = "accuracy"

args = TrainingArguments(
    f"checkpoints",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=15, # 10
    per_device_eval_batch_size=15, # 4
    num_train_epochs=15, # 3
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
    save_total_limit=5,
)

In [16]:
from datasets import load_metric
from sklearn.metrics import accuracy_score
import numpy as np
import evaluate
import torch

# def compute_metrics(eval_pred):
#     predictions, labels = eval_pred
#     predictions = np.argmax(predictions, axis=1)
#     return dict(accuracy=accuracy_score(predictions, labels))

# def compute_metrics(eval_preds):
#     metric = evaluate.load("glue", "mrpc")
#     logits, labels = eval_preds
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)



In [17]:
import torch

trainer = Trainer(
    model,
    args,
    train_dataset=train_set,
    eval_dataset=val_set,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

In [18]:
print(type(model))

<class 'transformers.models.beit.modeling_beit.BeitForImageClassification'>


## Train model

In [19]:
import wandb
from wandb.keras import WandbCallback

from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [20]:
from PIL import Image

Image.MAX_IMAGE_PIXELS = None  # 移除限制

In [21]:
train_results = trainer.train()

wandb: Currently logged in as: yusenluo (csci567). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.0
wandb: Run data is saved locally in /kaggle/working/wandb/run-20231124_094705-i5i7k95f
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run brisk-mountain-9
wandb: ⭐️ View project at https://wandb.ai/csci567/huggingface
wandb: 🚀 View run at https://wandb.ai/csci567/huggingface/runs/i5i7k95f


Epoch,Training Loss,Validation Loss,Accuracy
1,0.393700,0.341534,0.855548
2,0.284600,0.273377,0.887674
3,0.245700,0.288297,0.890864
4,0.194400,0.313538,0.880383
5,0.162300,0.341207,0.888585
6,0.141800,0.368806,0.881978
7,0.114000,0.428422,0.886762
8,0.105100,0.462692,0.888357
9,0.074500,0.554768,0.883117
10,0.070100,0.572783,0.886307


In [22]:
trainer.log_metrics("train", train_results.metrics)

***** train metrics *****
  epoch                    =          15.0
  total_flos               = 14247851306GF
  train_loss               =        0.1321
  train_runtime            =    3:54:50.18
  train_samples_per_second =        14.017
  train_steps_per_second   =         0.935


In [23]:
test_outputs = trainer.predict(test_set)

In [24]:
print(type(test_set))

<class 'datasets.arrow_dataset.Dataset'>


In [25]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

y_true = test_outputs.label_ids
y_pred = test_outputs.predictions.argmax(1)

print("acurracy", accuracy_score(y_true, y_pred))
print("precision", precision_score(y_true, y_pred, average='macro'))
print("recall", recall_score(y_true, y_pred, average='macro'))
print("f1_score", f1_score(y_true, y_pred, average='macro'))





acurracy 0.8947368421052632
precision 0.8880536932144042
recall 0.8896087769692467
f1_score 0.8888138893616722


## Save the model

In [26]:
pt_save_directory = "./pt_save_pretrained_ad"
processor.save_pretrained(pt_save_directory)
model.save_pretrained(pt_save_directory)

In [27]:
from transformers import AutoModelForSequenceClassification, BeitForImageClassification, AutoImageProcessor
from PIL import Image

pt_model = BeitForImageClassification.from_pretrained("./pt_save_pretrained_ad")
pt_processor = AutoImageProcessor.from_pretrained("./pt_save_pretrained_ad")

test_input_img_path = "/kaggle/input/csci-567/" + "7/62717.jpg"

 
test_input_img_PIL = Image.open(test_input_img_path)
print(test_input_img_PIL.format)

test_input_img = pt_processor(test_input_img_PIL, return_tensors="pt")
with torch.no_grad():
    logits = pt_model(**test_input_img).logits
    
# predicted_label = logits.argmax(-1).item()
print(logits.shape)
predicted_label = logits.argmax(-1)
print(predicted_label)

JPEG
torch.Size([1, 2])
tensor([1])


In [28]:
# from transformers import AutoImageProcessor, BeitForImageClassification
# import torch
# from datasets import load_dataset

# dataset = load_dataset("huggingface/cats-image")
# image = dataset["test"]["image"][0]

# image_processor = AutoImageProcessor.from_pretrained("microsoft/beit-base-patch16-224")
# model = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224")

# inputs = image_processor(image, return_tensors="pt")

# with torch.no_grad():
#     logits = model(**inputs).logits

# # model predicts one of the 1000 ImageNet classes
# predicted_label = logits.argmax(-1).item()
# print(model.config.id2label[predicted_label])

In [29]:
# labels = test_set.features['label'].names
labels = test_set.features['label_id']
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
# disp.plot()
# disp.plot(xticks_rotation=45)
print(test_outputs.predictions.shape)

(4389, 2)
